# Registration
This section introduces the following registration methods for point clouds, Registration is the task to estimate relative positions and poses between coordinate systems of data (point cloud, depth image, etc.). Examples that registration methods are effective are as follows.

- Estimation of the object pose: the registration method estimates a pose of a scanned point cloud with a pose of point cloud sampled from a CAD model.
- Integration of point clouds scanned on each location (the following figure): By applying the registration method to estimate a transformation matrix between scenes with overlap parts, we can integrate scene point cloud on a local coordinate system to the global coordinate system.

![fig](img/registration.png)

This section introduces the following registration methods for point clouds: 



- SVD (Singular Value Decomposition)
- ICP (Iterative Closest Point)
- Handcrafted feature and RANSAC


In [1]:
%load_ext autoreload
%autoreload 2

## Finding Transformation matrix with SVD (singular value decomposition)
This subsection show computing Transformation matrix with SVD. The subsection tutorial use following codes:


In [2]:
from tutlibs.sampling import furthest_point_sampling
from tutlibs.registration import estimate_transformation

import numpy as np
import inspect
from tutlibs.transformation import Transformation as tr
from tutlibs.transformation import TransformationMatrix as tm
from tutlibs.utils import single_color
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = tr.rotation(
    tr.translation(target_xyz, np.array([0.0, 0.1, 0.0])),
    'z',
    angle=30/180*np.pi
)

# down sampling
ds_indices = furthest_point_sampling(source_xyz, 1000)
source_ds_xyz = source_xyz[ds_indices]
target_ds_xyz = target_xyz[ds_indices]

# create correspondece set between source and target points
corr_set = np.tile(np.arange(len(source_ds_xyz))[:, np.newaxis], (1, 2)) # shape: (N, 2)

# use svd
trans_mat = estimate_transformation(source_ds_xyz, target_ds_xyz, corr_set=corr_set)
trans_source_xyz = tm.transformation(source_xyz, trans_mat)

# visualize results
obj_trans_source_points = jv.point(trans_source_xyz, single_color('#00ff00', len(trans_source_xyz)), [0, 255])
obj_target_points = jv.point(target_xyz, single_color('#ffff00', len(target_xyz)), [0, 255])
obj_source_points = jv.point(source_xyz, single_color('#ff0000', len(source_xyz)), [0, 255])
jv.display([obj_trans_source_points, obj_target_points, obj_source_points])

Output()

In the above output, red points are source points, yellow points are transformed points and green points are target points. (Yellow points overlap green points)

Next, we show `estimate_transformation` function that compute transformation matrix between source and target points.

In [10]:
print(inspect.getsource(estimate_transformation))

def estimate_transformation(
        source:np.ndarray, target:np.ndarray, corr_set:np.ndarray
        ) -> np.ndarray:
    """Estimate transformation with Singular Value Decomposition (SVD).

    Args:
        source: coordinates of source points, (N, 3)
        target: coordinates of target points, (M, 3)
        corr_set: correspondence index between source and target (L, 2)

    Returns:
        rotation_3x3: rotation matrix, (3, 3)
        translation_3: translation vector, (3)
    """

    source = source[corr_set[:, 0]]
    target = target[corr_set[:, 1]]

    centroid_source = np.mean(source, axis=0)
    centroid_target = np.mean(target, axis=0)

    source = source - centroid_source
    target = target - centroid_target

    correlation_mat = np.matmul(source.T, target)
    u, _, vh = np.linalg.svd(correlation_mat)
    rotation_3x3 = np.matmul(vh.T, u.T)
    translation_3 = centroid_target - np.matmul(rotation_3x3, centroid_source)

    return rotation_3x3, translation_3



## ICP (Iterative Closest Point)

This subsection demonstrates object registration with ICP for a object point cloud. The subsection tutorial use following codes:

In [5]:
from tutlibs.sampling import furthest_point_sampling
from tutlibs.registration import icp

import numpy as np
import inspect
from tutlibs.transformation import Transformation as tr
from tutlibs.transformation import TransformationMatrix as tm
from tutlibs.utils import single_color
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv

In [7]:
# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = tr.rotation(
    tr.translation(target_xyz, np.array([0.0, 0.1, 0.0])),
    'z',
    angle=30/180*np.pi
)

# down sampling
ds_indices = furthest_point_sampling(source_xyz, 1000)
source_ds_xyz = source_xyz[ds_indices]
target_ds_xyz = target_xyz[ds_indices]

# use ICP
trans_mat = icp(source_ds_xyz, target_ds_xyz, 20)
trans_source_xyz = tm.transformation(source_xyz, trans_mat)

# visualize results
obj_trans_source_points = jv.point(trans_source_xyz, single_color('#00ff00', len(trans_source_xyz)), [0, 255])
obj_target_points = jv.point(target_xyz, single_color('#ffff00', len(target_xyz)), [0, 255])
obj_source_points = jv.point(source_xyz, single_color('#ff0000', len(source_xyz)), [0, 255])
jv.display([obj_trans_source_points, obj_target_points, obj_source_points])

Output()

In the above output, red points are source points, yellow points are points estimated by ICP and green points are target points. (Yellow points overlap green points)

Next, we show `icp` function that estimate transformation matrix between source and target points.

In [13]:
print(inspect.getsource(icp))

def icp(source:np.ndarray, target:np.ndarray, iteration:int,
        threshold:float=0.0000001, init_trans_mat:np.ndarray=None) -> np.ndarray:
    """Iterative Closest Point

    Args:
        source: coordinates of source points, (N, 3)
        target: coordinates of target points, (M, 3)
        iteration: icp iteration
        threshold: convergence threshold
        init_trans_mat: initialinitial transformation matrix, (4, 4)

    Return:
        trans_mat: (4, 4)
    """

    if init_trans_mat is None:
        init_trans_mat = np.identity(4)
    trans_mat = init_trans_mat.copy() # transformation matrix

    target = target.copy()
    source = source.copy()

    for _ in range(iteration):
        # get previus transformed source xyz
        trans_source = tm.transformation_Nx3_with_4x4(source, trans_mat)

        # get correspondece set between previus target and transformed source xyz
        corr_set = brute_force_matching(trans_source, target)

        # get transformation estim

In the above output, red points are source points, yellow points are points estimated by ICP and green points are target points. (Yellow points overlap green points.)



## Handcrafted feature and RANSAC (RANdom SAmple Consensus)
This subsection demonstrates scene registration with handcrafted feature and RANSAC. The subsection tutorial use following code:

In [1]:
from tutlibs.sampling import furthest_point_sampling, voxel_grid_sampling
from tutlibs.registration import feature_ransac
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.normal_estimation import normal_estimation, normal_orientation

import numpy as np
from tutlibs.transformation import Transformation as tr
from tutlibs.transformation import TransformationMatrix as tm
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# data acquisition
target_xyz, _, data = io.read('../data/bunny_pc.ply')
target_normal = np.stack([data['nx'], data['ny'], data['nz']], axis=-1)
# target_normal = normal_estimation(target_xyz, k=15)

# source random transformation
rot_angle = 30/180*np.pi
source_xyz = tr.rotation(
    tr.translation(target_xyz, np.array([0.0, 0.1, 0.0])),
    'z',
    rot_angle
)

# down sampling (keypoint)
ds_indices = furthest_point_sampling(target_xyz, 1000)
source_ds_xyz = source_xyz[ds_indices]
target_ds_xyz = target_xyz[ds_indices]

# normals
target_ds_normal = target_normal[ds_indices]
source_ds_normal = tr.rotation(target_ds_normal, 'z', angle=rot_angle)

# extract Handcrafted feature
source_ds_pfh = pfh.compute(source_ds_xyz, source_ds_normal, 0.05)
target_ds_pfh = pfh.compute(target_ds_xyz, target_ds_normal, 0.05)

# feature matching
trans_mat = feature_ransac(source_ds_xyz, target_ds_xyz, source_ds_pfh,
                           target_ds_pfh, 3, 1000, 0.075)
trans_source_xyz = tm.transformation(source_xyz, trans_mat)

# visualize results
obj_trans_source_points = jv.point(trans_source_xyz, single_color('#00ff00', len(trans_source_xyz)), [0, 255])
obj_target_points = jv.point(target_xyz, single_color('#ffff00', len(target_xyz)), [0, 255])
obj_source_points = jv.point(source_xyz, single_color('#ff0000', len(source_xyz)), [0, 255])
jv.display([obj_trans_source_points, obj_target_points, obj_source_points])


Output()

In [3]:
# data acquisition
target_xyz, _, _ = io.read('../data/redwood_3dscan_1.ply')
target_ds_xyz = voxel_grid_sampling(target_xyz, 0.05)
target_ds_normal = normal_estimation(target_ds_xyz, k=15)
target_ds_normal = normal_orientation(target_ds_xyz, target_ds_normal)

# source random transformation
source_xyz, _, _ = io.read('../data/redwood_3dscan_2.ply')
source_ds_xyz = voxel_grid_sampling(source_xyz, 0.05)
source_ds_normal = normal_estimation(source_ds_xyz, k=15)
source_ds_normal = normal_orientation(source_ds_xyz, source_ds_normal)

# extract Handcrafted feature
source_ds_pfh = pfh.compute(source_ds_xyz, source_ds_normal, 0.25)
target_ds_pfh = pfh.compute(target_ds_xyz, target_ds_normal, 0.25)

# feature matching
trans_mat = feature_ransac(source_ds_xyz, target_ds_xyz, source_ds_pfh,
                           target_ds_pfh, 3, 100, 0.075)
trans_source_xyz = tm.transformation(source_ds_xyz, trans_mat)

# visualize results
obj_target_points = jv.point(target_ds_xyz, single_color('#00ff00', len(target_ds_xyz)))
obj_trans_source_points = jv.point(trans_source_xyz, single_color('#ffff00', len(trans_source_xyz)))
jv.display([obj_trans_source_points, obj_target_points])

Output()